In [1]:
import os
# перейдем в родительскую директорию чтобы использовать пакет raiflib без установки
notebook_path = os.getcwd()
os.chdir(os.path.dirname(notebook_path))

In [2]:
import pandas as pd
import numpy as np
from haversine import haversine, Unit
from geopy.geocoders import Nominatim

from tqdm import tqdm
from tqdm.auto import tqdm
tqdm.pandas()

In [3]:
df = pd.read_csv('../data/1.ipynb_Cleaning_data.csv', index_col=[0])
tdf = pd.read_csv('../data/1.ipynb_Cleaning_data_test.csv', index_col=[0])

# Добавление расстояния до столицы (Москва)

In [4]:
address = 'Москва'
geolocator = Nominatim(user_agent="Your_Name")
location = geolocator.reverse(f'55.750446 37.617494')

In [5]:
location.address

'Красное крыльцо, Соборная площадь, 19, Тверской район, Москва, Центральный федеральный округ, 103073, Россия'

In [6]:
# df_locations = df.progress_apply(lambda t: geolocator.reverse(f"{t['lat']} {t['lng']}").address, axis=1)

In [7]:
def get_distance(coords1, coords2):
    return haversine(coords1, coords2, Unit.KILOMETERS)

def get_location(address: str) -> float:
    geolocator = Nominatim(user_agent="Your_Name")
    location = geolocator.geocode(address)
    return location.latitude, location.longitude

moscow_coords = get_location('Москва')
df['distance_to_moscow'] = df.progress_apply(
                                lambda t: get_distance((t['lat'], t['lng']), moscow_coords), 
                                axis=1
)

tdf['distance_to_moscow'] = tdf.progress_apply(
                                lambda t: get_distance((t['lat'], t['lng']), moscow_coords), 
                                axis=1
)

100%|██████████| 2974/2974 [00:00<00:00, 18748.75it/s]


# Добавление расстояния до центра региона

In [8]:
# Сделаем Санкт-Петербург центром Ленинградской области, Москву центром московской
df['tmp_region'] = df['region'].where(df.city != 'Санкт-Петербург', 'Ленинградская область')
df['tmp_region'] = df['tmp_region'].where(df.city != 'Москва', 'Московская область')
tdf['tmp_region'] = tdf['region'].where(tdf.city != 'Санкт-Петербург', 'Ленинградская область')
tdf['tmp_region'] = tdf['tmp_region'].where(tdf.city != 'Москва', 'Московская область')

In [9]:
region_centers = df.groupby(['tmp_region', 'city'], as_index=False) \
                   .agg({'id':'count'}) \
                   .sort_values(by='id', ascending=False) \
                   .groupby('tmp_region').head(1) \
                   .rename(columns={
                       'id':'count',
                       'city': 'region_center'
                    })

region_centers['coords'] = region_centers.progress_apply(
                                lambda t: get_location(t['region_center']), 
                                axis=1
)
region_centers[['lat','lng']] = pd.DataFrame(region_centers['coords'].tolist(), index= region_centers.index)
region_centers = region_centers.drop(columns=['coords', 'count'])
region_centers

100%|██████████| 47/47 [00:23<00:00,  1.99it/s]


,tmp_region,region_center,lat,lng
974,Московская область,Москва,55.750446,37.617494
781,Ленинградская область,Санкт-Петербург,59.938732,30.316229
582,Краснодарский край,Краснодар,45.768401,39.026104
1483,Свердловская область,Екатеринбург,56.839104,60.608250
1182,Новосибирская область,Новосибирск,54.967814,82.951599
1838,Челябинская область,Челябинск,55.159841,61.402555
1419,Самарская область,Самара,53.198627,50.113987
1620,Татарстан,Казань,55.782355,49.124227
1719,Тюменская область,Тюмень,57.153534,65.542274
119,Башкортостан,Уфа,54.726141,55.947499


In [10]:
df = pd.merge(left=df, right=region_centers, on=['tmp_region'], how='left') \
    .rename(columns={
        'lat_x': 'lat',
        'lng_x': 'lng'
    })

df['distance_to_region_center'] = df.progress_apply(
                                lambda t: get_distance((t['lat'], t['lng']), (t['lat_y'], t['lng_y'])), 
                                axis=1
)

df = df.drop(columns=['lat_y', 'lng_y', 'region_center', 'tmp_region'])
# ----
tdf = pd.merge(left=tdf, right=region_centers, on=['tmp_region'], how='left') \
    .rename(columns={
        'lat_x': 'lat',
        'lng_x': 'lng'
    })

tdf['distance_to_region_center'] = tdf.progress_apply(
                                lambda t: get_distance((t['lat'], t['lng']), (t['lat_y'], t['lng_y'])), 
                                axis=1
)

tdf = tdf.drop(columns=['lat_y', 'lng_y', 'region_center', 'tmp_region'])

100%|██████████| 2974/2974 [00:00<00:00, 18121.34it/s]


# EDA числовых признаков

In [11]:
# Заметим, что переменная total_square имеет распределение, близкое к лог-нормальному
df['total_square'] = df.apply(lambda t: np.log(t['total_square']), axis=1)

tdf['total_square'] = tdf.total_square.apply(np.log)

In [12]:
tdf.to_csv('../data/2.ipynb_EDA_and_Feature_engineering_test.csv')
df.to_csv('../data/2.ipynb_EDA_and_Feature_engineering.csv')